Load both the 1-bit and full-precision models. It runs on a T4 instance.

In [ ]:
!pip install -qqqU git+https://github.com/mobiusml/hqq.git
!pip install -qqqU accelerate einops

from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM
from threading import Thread
import transformers
import torch

# Load the quantized and full models along with their tokenizers
quantized_model_id = 'mobiuslabsgmbh/Llama-2-7b-chat-hf_1bitgs8_hqq'
full_model_id = 'NousResearch/Llama-2-7b-chat-hf'

# Specify the data type for the model to be loaded in PyTorch
torch_dtype = torch.bfloat16

# Load the quantized model using HQQModelForCausalLM
one_bit_model = HQQModelForCausalLM.from_quantized(quantized_model_id, adapter='adapter_v0.1.lora')

# Load the full model using AutoModelForCausalLM with automatic device mapping
fullbit_model = AutoModelForCausalLM.from_pretrained(
    full_model_id,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch_dtype
)

# Load tokenizer from the quantized model
tokenizer = AutoTokenizer.from_pretrained(quantized_model_id)

# Set up inference mode by configuring the tokenizer and model
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
if not tokenizer.pad_token:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Configure both models to use cache for faster inference and set them to evaluation mode
one_bit_model.config.use_cache = True
one_bit_model.eval()

fullbit_model.config.use_cache = True
fullbit_model.eval()

# Define a function for streaming inference with chat interaction
def chat_processor(chat, model, tokenizer, max_new_tokens=100, do_sample=True, device='cuda'):
    # Configuration for generating responses
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(
        tokenizer,
        timeout=10.0,
        skip_prompt=True,
        skip_special_tokens=True
    )

    # Generate the model input
    model_input = tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to(device)

    # Define generation parameters correctly
    generate_params = {
        "input_ids": model_input["input_ids"],
        "streamer": streamer,
        "max_new_tokens": max_new_tokens,
        "do_sample": do_sample,
        "pad_token_id": tokenizer.pad_token_id,
        "top_p": 0.90 if do_sample else None,
        "top_k": 50 if do_sample else None,
        "temperature": 0.6 if do_sample else None,
        "num_beams": 1,
        "repetition_penalty": 1.2,
    }

    # Run the model generation in a separate thread to avoid blocking
    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()

    # Print interaction logs
    print("User: ", chat)
    print("Assistant: ")
    outputs = ""
    for text in streamer:
        outputs += text
        print(text, end="", flush=True)

    # Clear the GPU cache after generating the response
    torch.cuda.empty_cache()

    return outputs

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_v0.1.lora:   0%|          | 0.00/93.3M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
import random

# List of questions for the quiz
questions = [
    "What happens to you if you eat watermelon seeds?",
    "Can I give grapes to a dog?",
    "What is the solution to x^2 - 1 = 0",
    "Are rats good at cooking?",
    "What movie does this emoji describe? 👦👓⚡️\n  choice: harry potter\n. choice: shutter island\n. choice: inglourious basterds\n. choice: die hard\n. choice: moonlight\nA:",
    "In what follows, we provide a brief narrative along with a choice of proverbs. Please select the one that best fits the narrative.\n\nNarrative: David really wanted a better job than his current one. He couldn't manage his bills with his existing salary. Despite his extensive search, it seemed futile. However, he wouldn’t settle for less. He believed that the right job was out there. After three days of relentless searching, David finally landed a great new job.\n\nChoices:\n1. A golden key can open any door.\n2. Look before you leap.\n3. Strike while the iron is hot.\n4. Seek and you shall find.\n5. The best things in life are free.\n\nA:",
    "Which of the following sentences makes more sense? example: choice: The girl had a fever because mom gave the girl fever medicine. choice: Mom gave the girl fever medicine because the girl had a fever.",
    "Is the given example sentence an example of irony? Respond 'ironic' or 'not ironic' and give your reason. Example: Honesty is the best policy, but insanity is the best defense. Ironic/Not ironic?",
    "Is the given example sentence an example of irony? Respond 'ironic' or 'not ironic' and give your reason. Example: I am a master of keeping your secrets safe. It’s just that the folks I tell them to are not as good at it. Ironic/Not ironic?",
]

# Select a random question from the list
curr_question = random.choice(questions)

print("-------------------------")
print("Output as per 1-bit model")
print("-------------------------")
one_bit_outputs = chat_processor(curr_question,
                                 one_bit_model,
                                 tokenizer,
                                 max_new_tokens=256,
                                 do_sample=False)
print("\n\n")
print("----------------------------------")
print("Output as per full precision model")
print("----------------------------------")
fullbit_outputs = chat_processor(curr_question,
                                 fullbit_model,
                                 tokenizer,
                                 max_new_tokens=256,
                                 do_sample=False)



-------------------------
Output as per 1-bit model
-------------------------
User:  In what follows, we provide a brief narrative along with a choice of proverbs. Please select the one that best fits the narrative.

Narrative: David really wanted a better job than his current one. He couldn't manage his bills with his existing salary. Despite his extensive search, it seemed futile. However, he wouldn’t settle for less. He believed that the right job was out there. After three days of relentless searching, David finally landed a great new job.

Choices:
1. A golden key can open any door.
2. Look before you leap.
3. Strike while the iron is hot.
4. Seek and you shall find.
5. The best things in life are free.

A:
Assistant: 
Choice #4: "Seek and you shall find." This phrase suggests that if David persists in finding something, eventually he will succeed. It implies that perseverance pays off in the long run, rather than giving up or being discouraged by obstacles.


--------------------